In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [2]:
import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [11]:
year = 2022
month = 2
df = read_data(f's3://dtc-mlops/data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [9]:
import numpy as np
np.std(y_pred)

5.28140357655334

In [12]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [25]:
output_file = f'output/yellow_{year:04d}-{month:02d}.parquet'
df_result = pd.DataFrame()
df_result['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df_result['y_pred'] = y_pred
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [26]:
!ls -al output/

total 58600
drwxrwxr-x 2 ubuntu ubuntu     4096 Jun 17 09:31 .
drwxrwxr-x 3 ubuntu ubuntu     4096 Jun 17 09:31 ..
-rw-rw-r-- 1 ubuntu ubuntu 59994831 Jun 17 11:05 yellow_2022-02.parquet
